# Analysis of the Temporal Evolution of Loads by Interconnection

This notebook analyzes the time series of annual total and peak loads from TELL by interconnection.

In [2]:
# Start by importing the packages we need:
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


## Set the Directory Structure

In [14]:
# Identify the top-level directory and the subdirectory where the data will be stored:
load_data_input_dir =  '/Users/burl878/Documents/IMMM/GODEEP/Data/TELL/Production_Runs/tell_data/'
data_output_dir =  '/Users/burl878/Documents/Code/code_repos/load_analysis/data/'
image_output_dir =  '/Users/burl878/Documents/Code/code_repos/load_analysis/figures/'


## Process the TELL Output Files

In [4]:
# Read in the BA-to-Interconnection mapping file:
ba_mapping = pd.read_csv(data_output_dir + 'BA_to_Interconnection_Mapping.csv')


In [11]:
# Check to see if the processed output file exist and if not then create it:
if os.path.isfile((os.path.join(data_output_dir, 'Interconnection_Non_Transportation_Load_Time_Series.csv'))) == False:

   # Set the scenarios to process:
   scenarios = ['historic', 'BAU_Climate', 'NetZeroNoCCS_Climate']
    
   #Initiate a counter and empty dataframe to store the results:
   counter = 0;
   output_df = pd.DataFrame()

   # Loop over the scenarios and process each time series:
   for s in range(len(scenarios)):
       if scenarios[s] == 'historic':
          start_year = 1980; end_year = 2020; interval = 1
       else:
          start_year = 2020; end_year = 2051; interval = 5
        
       # Loop over the years from the start_year to the end_year in 5-year increments:
       for year in range(start_year,end_year,interval):
           # Read in the TELL BA output file for that year and scenario:
           if scenarios[s] == 'historic':
              tell_df = pd.read_csv(load_data_input_dir + 'outputs/tell_output/historic/' + str(year) + '/TELL_Balancing_Authority_Hourly_Load_Data_' + str(year) + '_Scaled_' + str(year) + '.csv')
           if scenarios[s] == 'BAU_Climate':
              tell_df = pd.read_csv(load_data_input_dir + 'outputs/tell_output/BAU_Climate/' + str(year) + '/TELL_Balancing_Authority_Hourly_Load_Data_' + str(year) + '_Scaled_' + str(year) + '.csv')
           if scenarios[s] == 'NetZeroNoCCS_Climate':
              tell_df = pd.read_csv(load_data_input_dir + 'outputs/tell_output/NetZeroNoCCS_Climate/' + str(year) + '/TELL_Balancing_Authority_Hourly_Load_Data_' + str(year) + '_Scaled_' + str(year) + '.csv')
             
           # Merge in the interconnection mapping using common BA codes:
           merged_df = pd.merge(tell_df, ba_mapping, on='BA_Code')
            
           # Sum the BA-level hourly loads into annual interconnection-level hourly loads:
           merged_df['Interconnection_Load_TWh'] = merged_df.groupby(['Interconnection', 'Time_UTC'])['Scaled_TELL_BA_Load_MWh'].transform('sum')    
            
           # Only keep the columns we need and subset to the unique values:
           merged_df = merged_df[['Time_UTC', 'Interconnection', 'Interconnection_Load_TWh']].drop_duplicates()
            
           # Make a list of all of the interconnections in the 'merged_df':
           ics = merged_df['Interconnection'].unique()
         
           # Loop over the interconnections and calculate the total and peak load for each year:
           for i in range(len(ics)):
               # Iterate the counter by one:
               counter = counter + 1
            
               # Subset to just the data for the interconnection being processed:
               subset_df = merged_df[merged_df['Interconnection'].isin([ics[i]])].copy()
          
               # Put the output in a new dataframe:
               output_df.loc[counter, 'Scenario'] = scenarios[s]
               output_df.loc[counter, 'Year'] = int(year)
               output_df.loc[counter, 'Interconnection'] = ics[i]
               output_df.loc[counter, 'Total_Load_TWh'] = (subset_df['Interconnection_Load_TWh'].sum().round(6)) / 1000000
               output_df.loc[counter, 'Peak_Load_MWh'] = subset_df['Interconnection_Load_TWh'].max()
            
               # Clean up
               del subset_df

           # Clean up
           del tell_df, merged_df, ics
         
   # Initiate an empty normalized load variable:
   output_df['Total_Load_Normalized'] = np.nan
   output_df['Peak_Load_Normalized'] = np.nan

   # Loop over the rows of the dataframe normalize the load by the base year value:
   for row in range(len(output_df)):
       # Find the total load value to normalize by:
       base_load = output_df['Total_Load_TWh'].loc[(output_df['Year'] == 2019) & 
                                                   (output_df['Interconnection'] == output_df['Interconnection'].iloc[row]) & 
                                                   (output_df['Scenario'] == 'historic')].values
    
       # Compute the normalized total load:
       output_df['Total_Load_Normalized'].iloc[row] = (output_df['Total_Load_TWh'].iloc[row] / base_load).round(3)
    
       # Find the peak load value to normalize by:
       peak_load = output_df['Peak_Load_MWh'].loc[(output_df['Year'] == 2019) & 
                                                  (output_df['Interconnection'] == output_df['Interconnection'].iloc[row]) & 
                                                  (output_df['Scenario'] == 'historic')].values
    
       # Compute the normalized peak load:
       output_df['Peak_Load_Normalized'].iloc[row] = (output_df['Peak_Load_MWh'].iloc[row] / peak_load).round(3)
    
       # Clean up
       del base_load, peak_load

   # Write out the dataframe to a .csv file:
   output_df.to_csv((os.path.join(data_output_dir, 'Interconnection_Non_Transportation_Load_Time_Series.csv')), sep=',', index=False)
   
else:
   # Read in the already processed output file:
   output_df = pd.read_csv((os.path.join(data_output_dir, 'Interconnection_Non_Transportation_Load_Time_Series.csv')))            
                  
# Preview the future dataframe:
output_df


,Scenario,Year,Interconnection,Total_Load_TWh,Peak_Load_MWh,Total_Load_Normalized,Peak_Load_Normalized
0,historic,1980.0,EIC,1115.357464,208687.28,0.376,0.381
1,historic,1980.0,WECC,268.160795,48710.28,0.363,0.369
2,historic,1980.0,ERCOT,95.371603,19833.71,0.312,0.323
3,historic,1981.0,EIC,1226.561061,227201.18,0.414,0.414
4,historic,1981.0,WECC,297.275794,51957.08,0.402,0.393
...,...,...,...,...,...,...,...
157,NetZeroNoCCS_Climate,2045.0,WECC,1376.280387,264961.06,1.862,2.007
158,NetZeroNoCCS_Climate,2045.0,ERCOT,653.514619,131840.13,2.136,2.149
159,NetZeroNoCCS_Climate,2050.0,EIC,6504.045138,1261883.28,2.195,2.301
160,NetZeroNoCCS_Climate,2050.0,WECC,1674.153149,319506.57,2.264,2.420


In [45]:
# Define a function to plot the time series of annual demand for each interconnection:
def plot_interconnection_load_time_series(data_input_dir: str, image_output_dir: str, image_resolution: int, save_images=False):
    
    # Read in the output file:
    output_df = pd.read_csv((os.path.join(data_output_dir, 'Interconnection_Non_Transportation_Load_Time_Series.csv')))
    
    # Subset to the time series for each interconnection and scenario:
    eic_historic = output_df[output_df['Scenario'].isin(['historic']) & output_df['Interconnection'].isin(['EIC'])]
    eic_bau = output_df[output_df['Scenario'].isin(['BAU_Climate']) & output_df['Interconnection'].isin(['EIC'])]
    eic_nz = output_df[output_df['Scenario'].isin(['NetZeroNoCCS_Climate']) & output_df['Interconnection'].isin(['EIC'])]
    
    wecc_historic = output_df[output_df['Scenario'].isin(['historic']) & output_df['Interconnection'].isin(['WECC'])]
    wecc_bau = output_df[output_df['Scenario'].isin(['BAU_Climate']) & output_df['Interconnection'].isin(['WECC'])]
    wecc_nz = output_df[output_df['Scenario'].isin(['NetZeroNoCCS_Climate']) & output_df['Interconnection'].isin(['WECC'])]
        
    ercot_historic = output_df[output_df['Scenario'].isin(['historic']) & output_df['Interconnection'].isin(['ERCOT'])]
    ercot_bau = output_df[output_df['Scenario'].isin(['BAU_Climate']) & output_df['Interconnection'].isin(['ERCOT'])]
    ercot_nz = output_df[output_df['Scenario'].isin(['NetZeroNoCCS_Climate']) & output_df['Interconnection'].isin(['ERCOT'])]
        
    # Make the raw load plots:
    plt.figure(figsize=(24, 12))
    plt.rcParams['font.size'] = 14
    
    plt.subplot(231)
    plt.plot(eic_historic['Year'], eic_historic['Total_Load_TWh'], color='black', linestyle='-', label='Historic', linewidth=2)
    plt.plot(eic_bau['Year'], eic_bau['Total_Load_TWh'], color='red', linestyle='-', label='BAU_Climate', linewidth=2)
    plt.plot(eic_nz['Year'], eic_nz['Total_Load_TWh'], color='blue', linestyle='-', label='NetZeroNoCCS_Climate', linewidth=2)
    plt.xlim([1980, 2050]); plt.xticks([1980, 1990, 2000, 2010, 2020, 2030, 2040, 2050],['1980','','2000','','2020','','2040',''])
    plt.legend(loc='upper left', prop={'size': 12})
    plt.ylabel('Annual Total Load [TWh]')
    plt.grid(True)
    plt.title('EIC Non-Transportation Load Projections')
    plt.title('a)', loc='left', fontsize=13)
    
    plt.subplot(232)
    plt.plot(wecc_historic['Year'], wecc_historic['Total_Load_TWh'], color='black', linestyle='-', label='Historic', linewidth=2)
    plt.plot(wecc_bau['Year'], wecc_bau['Total_Load_TWh'], color='red', linestyle='-', label='BAU_Climate', linewidth=2)
    plt.plot(wecc_nz['Year'], wecc_nz['Total_Load_TWh'], color='blue', linestyle='-', label='NetZeroNoCCS_Climate', linewidth=2)
    plt.xlim([1980, 2050]); plt.xticks([1980, 1990, 2000, 2010, 2020, 2030, 2040, 2050],['1980','','2000','','2020','','2040',''])
    plt.ylabel('Annual Total Load [TWh]')
    plt.grid(True)
    plt.title('WECC Non-Transportation Load Projections')
    plt.title('b)', loc='left', fontsize=13)
    
    plt.subplot(233)
    plt.plot(ercot_historic['Year'], ercot_historic['Total_Load_TWh'], color='black', linestyle='-', label='Historic', linewidth=2)
    plt.plot(ercot_bau['Year'], ercot_bau['Total_Load_TWh'], color='red', linestyle='-', label='BAU_Climate', linewidth=2)
    plt.plot(ercot_nz['Year'], ercot_nz['Total_Load_TWh'], color='blue', linestyle='-', label='NetZeroNoCCS_Climate', linewidth=2)
    plt.xlim([1980, 2050]); plt.xticks([1980, 1990, 2000, 2010, 2020, 2030, 2040, 2050],['1980','','2000','','2020','','2040',''])
    plt.ylabel('Annual Total Load [TWh]')
    plt.grid(True)
    plt.title('ERCOT Non-Transportation Load Projections')
    plt.title('c)', loc='left', fontsize=13)
    
    plt.subplot(234)
    plt.plot(eic_historic['Year'], eic_historic['Peak_Load_MWh']*0.001, color='black', linestyle='-', label='Historic', linewidth=2)
    plt.plot(eic_bau['Year'], eic_bau['Peak_Load_MWh']*0.001, color='red', linestyle='-', label='BAU_Climate', linewidth=2)
    plt.plot(eic_nz['Year'], eic_nz['Peak_Load_MWh']*0.001, color='blue', linestyle='-', label='NetZeroNoCCS_Climate', linewidth=2)
    plt.xlim([1980, 2050]); plt.xticks([1980, 1990, 2000, 2010, 2020, 2030, 2040, 2050],['1980','','2000','','2020','','2040',''])
    plt.xlabel('Year')
    plt.ylabel('Annual Peak Demand [GW]')
    plt.grid(True)
    plt.title('EIC Peak Demand Projections')
    plt.title('d)', loc='left', fontsize=13)
    
    plt.subplot(235)
    plt.plot(wecc_historic['Year'], wecc_historic['Peak_Load_MWh']*0.001, color='black', linestyle='-', label='Historic', linewidth=2)
    plt.plot(wecc_bau['Year'], wecc_bau['Peak_Load_MWh']*0.001, color='red', linestyle='-', label='BAU_Climate', linewidth=2)
    plt.plot(wecc_nz['Year'], wecc_nz['Peak_Load_MWh']*0.001, color='blue', linestyle='-', label='NetZeroNoCCS_Climate', linewidth=2)
    plt.xlim([1980, 2050]); plt.xticks([1980, 1990, 2000, 2010, 2020, 2030, 2040, 2050],['1980','','2000','','2020','','2040',''])
    plt.xlabel('Year')
    plt.ylabel('Annual Peak Demand [GW]')
    plt.grid(True)
    plt.title('WECC Peak Demand Projections')
    plt.title('e)', loc='left', fontsize=13)
    
    plt.subplot(236)
    plt.plot(ercot_historic['Year'], ercot_historic['Peak_Load_MWh']*0.001, color='black', linestyle='-', label='Historic', linewidth=2)
    plt.plot(ercot_bau['Year'], ercot_bau['Peak_Load_MWh']*0.001, color='red', linestyle='-', label='BAU_Climate', linewidth=2)
    plt.plot(ercot_nz['Year'], ercot_nz['Peak_Load_MWh']*0.001, color='blue', linestyle='-', label='NetZeroNoCCS_Climate', linewidth=2)
    plt.xlim([1980, 2050]); plt.xticks([1980, 1990, 2000, 2010, 2020, 2030, 2040, 2050],['1980','','2000','','2020','','2040',''])
    plt.xlabel('Year')
    plt.ylabel('Annual Peak Demand [GW]')
    plt.grid(True)
    plt.title('ERCOT Peak Demand Projections')
    plt.title('f)', loc='left', fontsize=13)
    
    # If the "save_images" flag is set to true then save the plot to a .png file:
    if save_images == True:
       filename = ('Interconnection_Load_Projections.png')
       plt.savefig(os.path.join(image_output_dir, filename), dpi=image_resolution, bbox_inches='tight', facecolor='white')
       plt.close()
    
            
    # Make the normalized load plots:
    plt.figure(figsize=(24, 12))
    plt.rcParams['font.size'] = 14
    
    plt.subplot(231)
    plt.plot(eic_historic['Year'], eic_historic['Total_Load_Normalized'], color='black', linestyle='-', label='Historic', linewidth=2)
    plt.plot(eic_bau['Year'], eic_bau['Total_Load_Normalized'], color='red', linestyle='-', label='BAU_Climate', linewidth=2)
    plt.plot(eic_nz['Year'], eic_nz['Total_Load_Normalized'], color='blue', linestyle='-', label='NetZeroNoCCS_Climate', linewidth=2)
    plt.xlim([1980, 2050]); plt.xticks([1980, 1990, 2000, 2010, 2020, 2030, 2040, 2050],['1980','','2000','','2020','','2040',''])
    plt.ylim([0, 3.5])
    plt.legend(loc='upper left', prop={'size': 12})
    plt.ylabel('Annual Total Load Normalized to 2019')
    plt.grid(True)
    plt.title('EIC Non-Transportation Load Projections')
    plt.title('a)', loc='left', fontsize=13)
    
    plt.subplot(232)
    plt.plot(wecc_historic['Year'], wecc_historic['Total_Load_Normalized'], color='black', linestyle='-', label='Historic', linewidth=2)
    plt.plot(wecc_bau['Year'], wecc_bau['Total_Load_Normalized'], color='red', linestyle='-', label='BAU_Climate', linewidth=2)
    plt.plot(wecc_nz['Year'], wecc_nz['Total_Load_Normalized'], color='blue', linestyle='-', label='NetZeroNoCCS_Climate', linewidth=2)
    plt.xlim([1980, 2050]); plt.xticks([1980, 1990, 2000, 2010, 2020, 2030, 2040, 2050],['1980','','2000','','2020','','2040',''])
    plt.ylim([0, 3.5])
    plt.ylabel('Annual Total Load Normalized to 2019')
    plt.grid(True)
    plt.title('WECC Non-Transportation Load Projections')
    plt.title('b)', loc='left', fontsize=13)
    
    plt.subplot(233)
    plt.plot(ercot_historic['Year'], ercot_historic['Total_Load_Normalized'], color='black', linestyle='-', label='Historic', linewidth=2)
    plt.plot(ercot_bau['Year'], ercot_bau['Total_Load_Normalized'], color='red', linestyle='-', label='BAU_Climate', linewidth=2)
    plt.plot(ercot_nz['Year'], ercot_nz['Total_Load_Normalized'], color='blue', linestyle='-', label='NetZeroNoCCS_Climate', linewidth=2)
    plt.xlim([1980, 2050]); plt.xticks([1980, 1990, 2000, 2010, 2020, 2030, 2040, 2050],['1980','','2000','','2020','','2040',''])
    plt.ylim([0, 3.5])
    plt.ylabel('Annual Total Load Normalized to 2019')
    plt.grid(True)
    plt.title('ERCOT Non-Transportation Load Projections')
    plt.title('c)', loc='left', fontsize=13)
    
    plt.subplot(234)
    plt.plot(eic_historic['Year'], eic_historic['Peak_Load_Normalized'], color='black', linestyle='-', label='Historic', linewidth=2)
    plt.plot(eic_bau['Year'], eic_bau['Peak_Load_Normalized'], color='red', linestyle='-', label='BAU_Climate', linewidth=2)
    plt.plot(eic_nz['Year'], eic_nz['Peak_Load_Normalized'], color='blue', linestyle='-', label='NetZeroNoCCS_Climate', linewidth=2)
    plt.xlim([1980, 2050]); plt.xticks([1980, 1990, 2000, 2010, 2020, 2030, 2040, 2050],['1980','','2000','','2020','','2040',''])
    plt.ylim([0, 3.5])
    plt.xlabel('Year')
    plt.ylabel('Annual Peak Demand Normalized to 2019')
    plt.grid(True)
    plt.title('EIC Peak Demand Projections')
    plt.title('d)', loc='left', fontsize=13)
    
    plt.subplot(235)
    plt.plot(wecc_historic['Year'], wecc_historic['Peak_Load_Normalized'], color='black', linestyle='-', label='Historic', linewidth=2)
    plt.plot(wecc_bau['Year'], wecc_bau['Peak_Load_Normalized'], color='red', linestyle='-', label='BAU_Climate', linewidth=2)
    plt.plot(wecc_nz['Year'], wecc_nz['Peak_Load_Normalized'], color='blue', linestyle='-', label='NetZeroNoCCS_Climate', linewidth=2)
    plt.xlim([1980, 2050]); plt.xticks([1980, 1990, 2000, 2010, 2020, 2030, 2040, 2050],['1980','','2000','','2020','','2040',''])
    plt.ylim([0, 3.5])
    plt.xlabel('Year')
    plt.ylabel('Annual Peak Demand Normalized to 2019')
    plt.grid(True)
    plt.title('WECC Peak Demand Projections')
    plt.title('e)', loc='left', fontsize=13)
    
    plt.subplot(236)
    plt.plot(ercot_historic['Year'], ercot_historic['Peak_Load_Normalized'], color='black', linestyle='-', label='Historic', linewidth=2)
    plt.plot(ercot_bau['Year'], ercot_bau['Peak_Load_Normalized'], color='red', linestyle='-', label='BAU_Climate', linewidth=2)
    plt.plot(ercot_nz['Year'], ercot_nz['Peak_Load_Normalized'], color='blue', linestyle='-', label='NetZeroNoCCS_Climate', linewidth=2)
    plt.xlim([1980, 2050]); plt.xticks([1980, 1990, 2000, 2010, 2020, 2030, 2040, 2050],['1980','','2000','','2020','','2040',''])
    plt.ylim([0, 3.5])
    plt.xlabel('Year')
    plt.ylabel('Annual Peak Demand Normalized to 2019')
    plt.grid(True)
    plt.title('ERCOT Peak Demand Projections')
    plt.title('f)', loc='left', fontsize=13)
    
    # If the "save_images" flag is set to true then save the plot to a .png file:
    if save_images == True:
       filename = ('Interconnection_Load_Projections_Normalized.png')
       plt.savefig(os.path.join(image_output_dir, filename), dpi=image_resolution, bbox_inches='tight', facecolor='white')
       plt.close()
    

In [46]:
plot_interconnection_load_time_series(data_input_dir = data_output_dir, 
                                      image_output_dir = image_output_dir, 
                                      image_resolution = 300, 
                                      save_images = True)
